# Scenario modelling

## Set up environment

In [1]:
CM_BASEPATH = '../cibusmod'

import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), CM_BASEPATH))

In [2]:
import CIBUSmod as cm
import CIBUSmod.utils.plot as plot

import time
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import cvxpy

In [3]:
from CIBUSmod.utils.misc import inv_dict, aggregate_data_coords_pair
from CIBUSmod.optimisation.indexed_matrix import IndexedMatrix
from CIBUSmod.optimisation.utils import make_cvxpy_constraint
from itertools import product

ImportError: cannot import name 'aggregate_data_coords_pair' from 'CIBUSmod.utils.misc' (/Users/wilhelmwanecek/dev/exjobb/eitm01-scenarios/../cibusmod/CIBUSmod/utils/misc.py)

In [ ]:
# Create session
session = cm.Session(
    name = 'main-mac',
    data_path = CM_BASEPATH + "/data",
    data_path_default = CM_BASEPATH + "/data/default",
    data_path_scenarios = "./workbooks"
)

# Load scenarios
session.add_scenario(
    "SCN_BASE",
    years=[2020],
    pars = "all",
    scenario_workbooks="base"
)

session.add_scenario(
    "BASELINE",
    years=[2020],
    pars = "all",
    scenario_workbooks="default_fix"
)

In [ ]:
%%time

scn = "BASELINE"

retrievers = {
    'Regions': cm.ParameterRetriever('Regions'),
    'DemandAndConversions': cm.ParameterRetriever('DemandAndConversions'),
    'CropProduction': cm.ParameterRetriever('CropProduction'),
    'FeedMgmt': cm.ParameterRetriever('FeedMgmt'),
    'GeoDistributor': cm.ParameterRetriever('GeoDistributor'),
}

cm.ParameterRetriever.update_all_parameter_values(**session[scn], year=2020)

# Instatiate Regions
regions = cm.Regions(
    par = retrievers['Regions'],
)

# Instantiate DemandAndConversions
demand = cm.DemandAndConversions(
    par = retrievers['DemandAndConversions'],
)

# Instantiate CropProduction
crops = cm.CropProduction(
    par = retrievers['CropProduction'],
    index = regions.data_attr.get('x0_crops').index
)

# Instantiate AnimalHerds
# Each AnimalHerd object is stored in an indexed pandas.Series
herds = cm.make_herds(regions)

# Instantiate feed management
feed_mgmt = cm.FeedMgmt(
    herds = herds,
    par = retrievers['FeedMgmt'],
)

# Instantiate geo distributor
optproblem = cm.FeedDistributor(
    regions = regions,
    demand = demand,
    crops = crops,
    herds = herds,
    feed_mgmt = feed_mgmt,
    par = retrievers['GeoDistributor'],
)

self = optproblem

In [ ]:
cm.ParameterRetriever.update_all_parameter_values()
cm.ParameterRetriever.update_relation_tables()

cm.ParameterRetriever.update_all_parameter_values(**session[scn], year=2020)

regions.calculate()
demand.calculate()
crops.calculate()
for h in herds:
    h.calculate()

self.make(use_cons=[1, 2, 3, 4, 5, 6, 10, 11, 12, 14], verbose=True)

## Data debugging

### Ensure feed_to_prod matches feeds

In [ ]:
feed_to_prod_feeds = set(feed_mgmt.par.get_unique("feed", qry='parameter=="feed_to_prod"').tolist())
missing_feeds = set()
for herd in self.herds:
    missing_feeds.update(set(filter(
        lambda f: f not in feed_to_prod_feeds,
        herd.par.get_unique("feed")
    )))
print(missing_feeds)

### List any crop/by products that are 100% imported

In [ ]:
imported = {}
keys = ["crop_prod", "by_prod"]
for bp_or_cp in keys:
    feed_to_by_prod = self._get_feed_to_prod_factors(bp_or_cp, index=True)
    _imported = feed_to_by_prod[feed_to_by_prod["share_domestic"] == 0][["feed_to_prod"]]
    imported[bp_or_cp] = list(_imported.index.unique("feed"))
    
set([x for k in keys for x in imported[k]])

## Improve numerics

In [ ]:
def print_ranges():
    for k, v in self.matrices().items():
        M = abs(v.M)
        M = M[M > 0]
        M_minmax = [M.min(), M.max()]
        e_min, e_max = np.log10(M_minmax)
        rng = e_max-e_min
        if rng>4:
            print(f"{k}: {int(rng):,} {tuple(M_minmax)}")

print("RANGES BEFORE:")
print_ranges()

# "We recommend that you scale the matrix coefficients so that their range 
# is contained in six orders of magnitude or less, and hopefully within [1e-3, 1e6]."

# AND:
# rhs should be on the order of 1e4 or less
# optimal value is less than 1e4


def rescale_constraints():
    constraints_to_scale = filter(
        lambda cons_label: any([c_nr in cons_label for c_nr in ["C1", "C12"]]),
        self.constraints.keys()
    )
    for cons_label in constraints_to_scale:
        cons = self.constraints[cons_label]
        for par_k in cons["pars"].keys():
            v = cons["pars"][par_k]
            if hasattr(v, 'M'):
                v.M /= (max(abs(v.M.max()), abs(v.M.min())) / 100)
            else:
                cons["pars"][par_k] /= (max(abs(v.max()), abs(v.min())) / 100)
    

In [ ]:
def improve_numerics(self):
    from CIBUSmod.optimisation.feed_dist import IndexedMatrix
   
    for name, C in self.constraints.items():
        M = [obj for obj in C['pars'].values() if isinstance(obj, IndexedMatrix)]
        assert len(M) == 1, "Expected one and only one IndexedMatrix"
        M = M[0]
        max_val = M.M.max()
        for name, obj in C['pars'].items():
            if isinstance(obj,IndexedMatrix):
                obj.M = obj.M / max_val
            else:
                obj[:] = obj / max_val
    print("Completed rescaling of matrices.")

improve_numerics(optproblem)

# Replace the objective function

While the original optimisation objective focused on minimising the change, we now instead want to maximize the protein contents.

## Mapping `x` to protein contents

First we need to create a row-array that maps each element in `x` with its protein content, so that we compute the aggregate protein amount from the decision variable.

In [ ]:
PROTEIN_CONTENTS = {
    "meat": 155.5,
    "milk": 35.0,
}

# Convert to thousands of prot. / kg, instead of straight
for k in PROTEIN_CONTENTS.keys():
    PROTEIN_CONTENTS[k] /= 1e3

def make_protein_mask_ani():
    RELEVANT_ANIMAL_PRODUCTS = ["meat", "milk"]
    
    # Get row index from animal product demand vector (ps,sp,ap)
    row_idx = pd.MultiIndex.from_tuples(
        [
            ("conventional", "cattle", "meat"),
            ("conventional", "cattle", "milk"),
            ("organic", "cattle", "meat"),
            ("organic", "cattle", "milk"),
        ],
        names=["prod_system", "species", "animal_prod"]
    )

    # Get col index from animal herds (sp,br,ps,ss,re)
    col_idx = self.x_idx["ani"]

    # To store data and corresponding row/col numbers for constructing matrix
    val = []
    row_nr = []
    col_nr = []

    # Go through animal herds
    for herd in self.herds:
        sp = herd.species
        br = herd.breed
        ps = herd.prod_system
        ss = herd.sub_system

        if sp != "cattle":
            continue

        def get_uniq(col):
            return herd.data_attr.get("production").columns.unique(col)
        
        # Get all animal products that we are concerned with
        aps = set(get_uniq("animal_prod")) & set(RELEVANT_ANIMAL_PRODUCTS)
        opss = get_uniq("prod_system")
        
        for ap, ops in product(aps, opss):
            if (ops, sp, ap) not in row_idx:
                continue
        
            # Get production of animal product (ap) from output production system (ops) per head
            # of defining animal of species (sp) and breed (br) in production system (ps), sub system (ss)
            # and region (re)
            res = (
                herd.data_attr.get("production")
                .loc[:, (ops, slice(None), ap)]
                .sum(axis=1)
            ) * PROTEIN_CONTENTS[ap]
        
            if all(res == 0):
                continue
        
            val.extend(res)
            col_nr.extend([col_idx.get_loc((sp, br, ps, ss, re)) for re in res.index])
            row_nr.extend(np.zeros(len(res)))

    # Aggregate data_coords_pair to ensure that any overlapping values are summed rather than replace each other
    val, (row_nr, col_nr) = aggregate_data_coords_pair(val, row_nr, col_nr)

    # Create Compressed Sparse Column matrix
    return scipy.sparse.coo_array((val, (row_nr, col_nr)), shape=(1, len(col_idx))).tocsc()


def make_protein_mask():
    A_ani = make_protein_mask_ani()
    A_crp = scipy.sparse.csc_matrix((1, len(self.x_idx["crp"])))
    A_fds = scipy.sparse.csc_matrix((1, len(self.x_idx["fds"])))

    return scipy.sparse.hstack([A_ani, A_crp, A_fds], format="csc")

make_protein_mask()

### Quick validity check

Ensure we only have values in the protein map where we expect to, i.e. for the added crops and for cattle.

In [ ]:
df_ani = pd.DataFrame(make_protein_mask_ani(), columns=self.x_idx["ani"])

# Check that only cattle has values in the ani part of the protein mask
for sp in df_ani.columns.unique("species"):
    is_all_zeroes = (df_ani.loc[:,(sp, slice(None), slice(None), slice(None), slice(None))]==0).all().all()
    is_cattle = sp == "cattle"
    assert is_all_zeroes != is_cattle

## Construct and replace the `cvxpy.Problem`

In [ ]:
self.solve(
    apply_solution=True,
    verbose=True,
    solver_settings=[{
        "solver": "GUROBI",
        "reoptimize": True,
        "verbose": True,
    }]
)

In [ ]:
self.feed_mgmt.calculate()

In [ ]:
protein_amount = (make_protein_mask() @ (self.problem.variables()[0]).value)[0]
print(f"{protein_amount:e}")

# Plot results

In [ ]:
session.store(
    scn, 2020,
    demand, regions, crops, herds, optproblem
)

In [ ]:
cm.plot.bar(
    session.get_attr('c', 'area', {'crop': ['land_use',None], 'region':None}).iloc[0].unstack('crop'),
    group_levels='land_use'
)

plt.show()   

In [ ]:
cm.plot.bar(
    session.get_attr('a', 'heads', ['region','species']).iloc[0].unstack('species')
)
plt.show()